# Série 3

Dans cette série on va voir les certificats CA de certains sites et les regarder dans plus de détails. 

# Exercice 1

On va faire quelques requêtes SSL vers des sites pour voir comment ils répondent.

## 1. Connaissance

La façon la plus simple de faire ces connexions est en fait d'utiliser le navigateur. 
Autant Mozilla Firefox, Chrome, Safari, Edge, vous permettent de montrer les certificats
utilisés pour sécuriser le site visité.

Donc pour cette première partie, visitez les sites suivants avec votre navigateur, et trouvez:
- le nombre de certificats dans la liste
- le certificat de base

Les sites sont:
- https://google.ch
- https://google.com
- https://moodle.unifr.ch/
- https://web.fledg.re

Est-ce que vous trouvez le site qui utilise Let's Encrypt?

## 2. Compréhension

Utiliser le navigateur est simple, mais en utilisant Python on voit un peu plus comment ça marche.
Dans l'exemple en bas, il y a quelques méthods qui ont été créées, tout d'abord ceux-ci:

- ´get_connection_chain´ se connecte au serveur indiqué et retourne la connection et la chaîne de certificats - ATTENTION: il faut seulement donner le nom de domaine, pas l'URL complète!
- ´dump_cert´ imprime un certificat
- ´dump_chain´ imprime toute la chaîne

Qu'est-ce que vous observez en lançant le code? Pourquoi c'est différent?

Grâce au Python, nous avons aussi de l'information supplémentaire sur la connexion. Essayez les deux adresses suivantes:
- https://xkcd.com
- https://sssscomic.com


In [3]:
# Exercice 1 - Partie 2

from OpenSSL import SSL, crypto
import socket, certifi

def dump_cert(cert):
    for component in cert.get_subject().get_components():
        print("Subject %s: %s" % (component))
             
    print("notBefore:", cert.get_notBefore())
    print("notAfter:", cert.get_notAfter())
    print("version:" + str(cert.get_version()))
    print("sigAlg:", cert.get_signature_algorithm())
    print("digest:", cert.digest('sha256'))
    print("issuer:", cert.get_issuer())
    print()
    
def get_connection_chain(host, port = 443):
    dst = (str.encode(host), port)
    ctx = SSL.Context(SSL.SSLv23_METHOD)
    s = socket.create_connection(dst)
    s = SSL.Connection(ctx, s)
    s.set_connect_state()
    s.set_tlsext_host_name(dst[0])

    s.sendall(b'HEAD / HTTP/1.0\n\n')
    s.recv(16)
    return (s, s.get_peer_cert_chain())

def dump_chain(chain):
    for pos, cert in enumerate(chain):
        print("Certificate #" + str(pos))
        dump_cert(cert)
        
def dump_connection(conn):
    print("Cipher name:", conn.get_cipher_name())
    print("Cipher size:", conn.get_cipher_bits())
    print("Cipher version:", conn.get_cipher_version())
    print("Cipher list:", conn.get_cipher_list())

conn, chain = get_connection_chain("google.ch")
dump_chain(chain)
dump_connection(conn)

Certificate #0
Subject b'C': b'US'
Subject b'ST': b'California'
Subject b'L': b'Mountain View'
Subject b'O': b'Google LLC'
Subject b'CN': b'*.google.ch'
notBefore: b'20210503115919Z'
notAfter: b'20210726115918Z'
version:2
sigAlg: b'sha256WithRSAEncryption'
digest: b'5C:F2:4C:8D:BF:22:3A:BF:54:79:B7:C7:49:17:DF:CA:63:EE:AF:3A:92:7D:90:72:36:87:F7:4F:E4:6D:0E:36'
issuer: <X509Name object '/C=US/O=Google Trust Services/CN=GTS CA 1O1'>

Certificate #1
Subject b'C': b'US'
Subject b'O': b'Google Trust Services'
Subject b'CN': b'GTS CA 1O1'
notBefore: b'20170615000042Z'
notAfter: b'20211215000042Z'
version:2
sigAlg: b'sha256WithRSAEncryption'
digest: b'95:C0:74:E3:59:02:A1:4A:BD:9D:19:AF:B6:E7:F8:0E:66:9F:F8:E2:36:32:70:53:9D:96:36:13:F0:4A:AA:21'
issuer: <X509Name object '/OU=GlobalSign Root CA - R2/O=GlobalSign/CN=GlobalSign'>

Cipher name: TLS_AES_256_GCM_SHA384
Cipher size: 256
Cipher version: TLSv1.3
Cipher list: ['TLS_AES_256_GCM_SHA384', 'TLS_CHACHA20_POLY1305_SHA256', 'TLS_AES_128_GCM

## 3. Application

Pour la troisième partie, on va enfin aller chercher le certificat de base. Comme on a vu dans la partie 2, le certificat de base n'est pas envoyé par le site. Ca donne du sens, parce qu'on ne fait à priori pas confiance au site. Donc on ne va pas accepter un certificat qui vient de là.

Il faut donc avoir une liste de certificats à qui on fait confiance. Cette liste réside sur votre ordinateur, et peut être récupérée avec ´certifi.where()´.

Vous pouvez maintenant utiliser la méthode ´get_root_cert´ sur un des certificats précédents pour vous retourner la chaîne de certificats de base.

Question:
- pourquoi on donne seulement le dernier élément de la liste (`chain[-1]`)?
- pourquoi on reçoit deux certificats à la fin?

In [5]:
# Exercice 1 - Partie 3

def get_root_chain(chain):
    store = crypto.X509Store()
    store.load_locations(certifi.where())
    store_ctx = crypto.X509StoreContext(store, chain[-2])
    return store_ctx.get_verified_chain()

dump_chain(get_root_chain(chain))


X509StoreContextError: [20, 0, 'unable to get local issuer certificate']